# DAG Workflow

This Notebook aim to show how create a DAG for workflow execution.
Each Task need to specify its own dependencies.
In the following lines the example that you can find in Argo documentation.

In [1]:
from argo.workflows.client.models import *

In [2]:
def get_dag():
    
    task_a = V1alpha1DAGTask(
        name="A", 
        template="echo", 
        arguments=V1alpha1Arguments(parameters=[V1alpha1Parameter(name="message", value="A")]))
    
    task_b = V1alpha1DAGTask(
        name="B", 
        template="echo", 
        arguments=V1alpha1Arguments(parameters=[V1alpha1Parameter(name="message", value="B")]), 
        dependencies=["A"])

    task_c = V1alpha1DAGTask(
        name="C",
        template="echo", 
        arguments=V1alpha1Arguments(parameters=[V1alpha1Parameter(name="message", value="C")]), 
        dependencies=["A"])

    task_d = V1alpha1DAGTask(
        name="D",
        template="echo", 
        arguments=V1alpha1Arguments(parameters=[V1alpha1Parameter(name="message", value="D")]), 
        dependencies=["B", "C"])

    dag = V1alpha1DAGTemplate(tasks=[
        task_a,
        task_b,
        task_c,
        task_d
    ])
    
    return dag

In [3]:
def get_workflow():
    
    spec = V1alpha1WorkflowSpec(
        entrypoint="diamond", 
        templates=[
            
            V1alpha1Template(
                name="echo", 
                container=V1Container(
                    name="echo", 
                    image="alpine:3.7", 
                    command=["echo", "{{inputs.parameters.message}}"]),
                inputs=V1alpha1Inputs(parameters=[V1alpha1Parameter(name="message")])),

            V1alpha1Template(name="diamond", dag=get_dag())

        ])

    workflow = V1alpha1Workflow(
        api_version="argoproj.io/v1alpha1",
        kind="Workflow",
        metadata=V1ObjectMeta(generate_name="dag-diamond-"), 
        spec=spec,
        status=V1alpha1WorkflowStatus())    

    return workflow


In [4]:
workflow = get_workflow()

In [6]:
from argo.workflows.client import V1alpha1Api
from argo.workflows.config import load_kube_config

load_kube_config()  # loads local configuration from ~/.kube/config

v1alpha1 = V1alpha1Api()

result = v1alpha1.create_namespaced_workflow('default', workflow)

In [15]:
# The following fragment must be invoked more time, until the workflow is completed its execution

from kubernetes.client.rest import ApiException
from kubernetes import client, config

config.load_kube_config()

wfs = v1alpha1.list_namespaced_workflows(namespace="default")
for job in wfs.items:
    if job.metadata.name == result.metadata.name:
        for pod_name in job.status.nodes.keys():
            if job.status.nodes[pod_name].type == 'Pod':
                try:
                    api_instance = client.CoreV1Api()
                    # Pod created by Argo has init containers, main container and wait container...
                    api_response = api_instance.read_namespaced_pod_log(name=pod_name, container="main", namespace='default')
                    print(f"Log for {pod_name}: {api_response}")
                except ApiException as e:
                    print(f'Found exception in reading the logs {pod_name}')

Log for dag-diamond-xl7vk-1591302304: A

Log for dag-diamond-xl7vk-1624857542: C

Log for dag-diamond-xl7vk-1641635161: B

Log for dag-diamond-xl7vk-1675190399: D

